# **Extract DayMET Data**

By Bridget Bittmann

Date created: March 28, 2022

Date editted: May 4, 2022

Purpose: This script extracts DayMET data from Google Earth Engine using the Earth Engine API in Python. This script also copied over a SSEBop ET data dataset and calculated zonal stats based on polygons. 

In [40]:
# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print('geemap package not installed. Installing ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

# Checks whether this notebook is running on Google Colab
try:
    import google.colab
    import geemap.eefolium as emap
except:
    import geemap as emap

# Authenticates and initializes Earth Engine
import ee

try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()  

In [41]:
!pip install geopandas
import geopandas as gpd #import independent shapefile
import json #for metadata of shapefile
import os #for file paths
import numpy as np #for stats and arrays
import pandas as pd #for dataframes

In [42]:
#Connect to Google Drive if you want to export images
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Go to correct folder in Google Drive
%cd drive/MyDrive/spatial_colab/datasets/
%ls

/content/drive/MyDrive/spatial_colab/datasets
2021_clip/      climate_stats/         IrrMapper/    output_files/
Boise_CDL/      diversion_timeseries/  LBRB_shp/     subset_test_shp/
CDL_reproject/  irrigation_companies/  lcmap_files/
CDL_subsets/    irrig_lbrb/            masked/


In [43]:
## ----------------------------------- ## 
## 1. Import shapefile to clip dataset ##
## ----------------------------------- ## 

shp_file = 'subset_test_shp/subset_class/subset_class.shp'
subset = emap.shp_to_ee(shp_file)

map=emap.Map(center=(43.6150, -116.2023),zoom=8)
map.addLayer(ee.Image().paint(subset, 0, 2), {}, 'POU')
map.addLayerControl()
map

In [ ]:
## --------------------------------------- ##
## 2. IMPORT THE DAYMET DATA FOR MAX TEMPS ##
## --------------------------------------- ##

years = np.arange(1985,2021)
mean_max = []
for i in range(len(years)):
  daymet = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filterDate((str(years[i])+'-06-01'), (str(years[i])+'-8-31')) #get image collection
  mxtmp = daymet.select('tmax').map(lambda image: image.clip(subset)).mean() #select the bands to analyze
  mean_max.append(mxtmp) #calculate the mean across all pixels

means_max_temp = ee.ImageCollection(mean_max) #convert list of image to image collection for zonal stats command

maximumTemperatureVis = {
  'min': -40.0,
  'max': 30.0,
  'palette': ['1621A2', 'white', 'cyan', 'green', 'yellow', 'orange', 'red'],
}

Map = emap.Map(center=(43.6150, -116.2023),zoom=8)
Map.addLayer(means_max_temp, maximumTemperatureVis, 'tmax')
Map

In [ ]:
## ------------------------------------------- ##
## 3. IMPORT THE DAYMET DATA FOR PRECIPITATION ##
## ------------------------------------------- ##

years = np.arange(1986,2022)
sum_pr = []
for i in years:
  daymet = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filterDate((str(i)+'-10-01'), (str((i+1))+'-9-30')) #get image collection
  prcp = daymet.select('prcp').map(lambda image: image.clip(subset)).sum() #select the bands to analyze
  sum_pr.append(prcp) #calculate the mean across all pixels

sum_precip = ee.ImageCollection(sum_pr) #convert list of image to image collection for zonal stats command

precip_vis = {
  'min': 0,
  'max': 544,
  'palette': ['1621A2', 'white', 'cyan', 'green', 'yellow', 'orange', 'red'],
}

Map = emap.Map(center=(43.6150, -116.2023),zoom=8)
Map.addLayer(sum_precip, precip_vis, 'prcp')
Map

In [ ]:
## ----------------------------- ##
## 4. IMPORT MONTHLY PRECIP DATA ##
## ----------------------------- ##

years = np.arange(1987,2021)
months = np.arange(3,11)
sum_month_pr = []

for i in years:
  for m in months:
    if (m == 3) or (m == 5) or (m == 7) or (m == 8) or (m==10):
      daymet = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filterDate((str(i)+'-'+str(m)+'-01'), (str(i)+'-'+str(m)+'-31')) #get image collection
      prcp = daymet.select('prcp').map(lambda image: image.clip(subset)).sum().set('system:index', (str(i)+'-'+str(m))) #select the bands to analyze
      sum_month_pr.append(prcp) #calculate the mean across all pixels
    else: 
      daymet = ee.ImageCollection("NASA/ORNL/DAYMET_V4").filterDate((str(i)+'-'+str(m)+'-01'), (str(i)+'-'+str(m)+'-30')) #get image collection
      prcp = daymet.select('prcp').map(lambda image: image.clip(subset)).sum().set('system:index', (str(i)+'-'+str(m))) #select the bands to analyze
      sum_month_pr.append(prcp) #calculate the mean across all pixels

sum_month_precip = ee.ImageCollection(sum_month_pr) #convert list of image to image collection for zonal stats command


In [70]:
## IMPORT MONTHLY ET DATA ##

et_irrig = []

# for y in years:
#   et_data = ee.ImageCollection('projects/earthengine-legacy/assets/users/bridgetbittmann/ssebop/boise').filterDate((str(y)+'-03-01'), str(y)+'-10-31')
#   et = et_data.map(lambda image: image.clip(subset)).sum().multiply(0.00001).set({'system:index': str(y)}) # sum and convert to meters 
#   et_irrig.append(et)

# et_irrig = ee.ImageCollection(et_irrig)

for y in years:
  et_data = ee.ImageCollection('projects/earthengine-legacy/assets/users/bridgetbittmann/ssebop/boise').filterDate((str(y)+'-04-01'), str(y)+'-9-30')
  et_march = ee.ImageCollection('projects/earthengine-legacy/assets/users/bridgetbittmann/ssebop/boise').filterDate((str(y)+'-03-01'), str(y)+'-03-31')
  et_march = et_march.map(lambda image: image.clip(subset)).sum().multiply(0.00001).set({'system:index': (str(y)+'-03')})
  et_oct = ee.ImageCollection('projects/earthengine-legacy/assets/users/bridgetbittmann/ssebop/boise').filterDate((str(y)+'-10-01'), str(y)+'-10-31')
  et_oct = et_oct.map(lambda image: image.clip(subset)).sum().multiply(0.00001).set({'system:index': (str(y)+'-10')})
  et = et_data.map(lambda image: image.clip(subset)).sum().multiply(0.00001).set({'system:index': str(y)}) # sum and convert to meters 
  et_irrig.append(et)
  et_irrig.append(et_march)
  et_irrig.append(et_oct)

et_cut = ee.ImageCollection(et_irrig)
  

In [71]:

## ------------------------ ##
## 4. CALCULATE ZONAL STATS ##
## ------------------------ ##

# Allowed output formats: csv, shp, json, kml, kmz
# Allowed statistics type: MEAN, MAXIMUM, MINIMUM, MEDIAN, STD, MIN_MAX, VARIANCE, SUM

# out_stats = os.path.join('climate_stats/maxtemp_stats.csv')
# emap.zonal_statistics(means_max_temp, subset, out_stats, statistics_type='MEAN', scale=1000)

# out_stats = os.path.join('climate_stats/precip_stats.csv')
# emap.zonal_statistics(sum_precip, subset, out_stats, statistics_type='MEAN', scale=1000)

# out_stats = os.path.join('climate_stats/month_precip_stats.csv')
# emap.zonal_statistics(sum_month_precip, subset, out_stats, statistics_type='MEAN', scale=1000)

# out_stats = os.path.join('climate_stats/et.csv')
# emap.zonal_statistics(et_irrig, subset, out_stats, statistics_type='MEAN', scale=30)

out_stats = os.path.join('climate_stats/et_cut.csv')
emap.zonal_statistics(et_cut, subset, out_stats, statistics_type='MEAN', scale=30)

Computing statistics ...
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/spatial_colab/datasets/climate_stats/et_cut.csv


In [54]:
## ---------------------------------------------- ##
## 5. CREATE CLIMATE STAT FOR EACH POU AND EXPORT ##
## ---------------------------------------------- ##

precip = pd.read_csv('climate_stats/precip_stats.csv')
max_temp = pd.read_csv('climate_stats/maxtemp_stats.csv')

names = precip['DIV_NAME_']
print(names)

for i in range(len(names)):
  df = pd.DataFrame(years, columns=['Year'])
  df['DIV_NAME'] = names[i]
  df['Precip_mm'] = precip.iloc[0,0:34].values
  df['Max_temp'] = max_temp.iloc[0,2:36].values
  out_path = os.path.join('climate_stats/final/'+names[i]+'_climate.csv')
  df.to_csv(out_path)


0                                     PioneerIrrigation
1                                      Ballentyne Ditch
2                   Eureka #2_Upper Center Point Feeder
3                     Farmers Cooperative Seebree Canal
4                                   Farmers Union Ditch
5     Middleton Irrigation Association and Middleton...
6                     New Dry Creek Canal_PD from River
7                                       Riverside Canal
8                                            Rossi Mill
9                    Settlers Irrigation District Ditch
10                                     Siebenberg Canal
Name: DIV_NAME_, dtype: object


In [101]:
## --------------- ## 
## COMPARE ET DATA ##
## --------------- ## 

et_whole = pd.read_csv('climate_stats/et.csv').drop(columns = ['Shape_Area', 'system:index', 'Shape_Le_1', 'Shape_Leng'], axis=1)
et_cut = pd.read_csv('climate_stats/et_cut.csv')

names = et_whole['DIV_NAME_']
et = []

march = pd.DataFrame(et_cut['DIV_NAME_'])
october = pd.DataFrame(et_cut['DIV_NAME_'])
cut = pd.DataFrame(et_cut['DIV_NAME_'])

for i in years:
  fill = str(i)
  march[fill] = et_cut[et_cut.columns[et_cut.columns == (str(i)+'-03_et')]]
  october[fill] = et_cut[et_cut.columns[et_cut.columns == (str(i)+'-10_et')]]
  cut[fill] = et_cut[et_cut.columns[et_cut.columns == (str(i)+'_et')]]

for i in range(len(names)):
  df = pd.DataFrame(years, columns=['Year'])
  df['DIV_NAME'] = names[i]
  df['et_whole'] = et_whole.iloc[0,0:34].values
  df['et_cut'] = cut.iloc[0,1:35].values
  df['march'] = march.iloc[0,1:35].values
  df['october'] = october.iloc[0,1:35].values
  et.append(df)

et = pd.concat(et)

et['diff'] = et['et_whole'] - et['et_cut']
display(et)


,Year,DIV_NAME,et_whole,et_cut,march,october,diff
0,1987,PioneerIrrigation,0.82531,0.728938,0.050834,0.045542,0.096372
1,1988,PioneerIrrigation,0.812668,0.761912,0.004939,0.045817,0.050756
2,1989,PioneerIrrigation,0.874915,0.807814,0.035176,0.032995,0.067101
3,1990,PioneerIrrigation,0.904566,0.826024,0.014621,0.06392,0.078541
4,1991,PioneerIrrigation,0.781241,0.734707,0.005623,0.040911,0.046534
...,...,...,...,...,...,...,...
29,2016,Siebenberg Canal,0.847029,0.697641,0.078659,0.070733,0.149388
30,2017,Siebenberg Canal,0.816669,0.688044,0.076202,0.052431,0.128626
31,2018,Siebenberg Canal,0.831782,0.703525,0.078271,0.049991,0.128257
32,2019,Siebenberg Canal,0.800255,0.667041,0.060636,0.072587,0.133214


In [ ]:
## ------------------------- ##
## COPY ET DATA to Cloud API ##
## ------------------------- ##

# folder from where to copy
src_folder = "projects/earthengine-legacy/assets/users/dgketchum/ssebop/boise"
# folder where to copy
dest_folder = "projects/ee-bridgetbittmann/assets/ssebop"

# get all assets in the folder
assets = ee.data.listAssets({'parent': src_folder})

# loop through assets and copy them one by one to the new destination
for asset in assets['assets']:
    # construct destination path
    new_asset = dest_folder + '/' + asset['id'].split('/')[-1]
    # copy to destination
    ee.data.copyAsset(asset['id'], new_asset, True)

In [ ]:
## ------------------------------------------- ##
## COPY ET DATA from Cloud API to Earth Engine ##
## ------------------------------------------- ##

# folder from where to copy
src_folder = "projects/ee-bridgetbittmann/assets/ssebop"
# folder where to copy
dest_folder = "projects/earthengine-legacy/assets/users/bridgetbittmann/ssebop/boise"

# get all assets in the folder
assets = ee.data.listAssets({'parent': src_folder})

# loop through assets and copy them one by one to the new destination
for asset in assets['assets']:
    # construct destination path
    new_asset = dest_folder + '/' + asset['id'].split('/')[-1]
    # copy to destination
    ee.data.copyAsset(asset['id'], new_asset, True)

In [ ]:
## -------------------------------------- ## 
## Calculate ET zonal stats based on POUs ## 
## -------------------------------------- ## 